# Problem 1: MLP Classifier

Import libraries

In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
import sklearn.neural_network
import sklearn.model_selection

Load in training and validation data

In [3]:
data_dir = os.path.abspath("data_fashion/")

train_x = pd.read_csv(os.path.join(data_dir, "x_train.csv")).to_numpy()
train_y = pd.read_csv(os.path.join(data_dir, "y_train.csv")).to_numpy()[:,1].astype(np.int32)

valid_x = pd.read_csv(os.path.join(data_dir, "x_valid.csv")).to_numpy()
valid_y = pd.read_csv(os.path.join(data_dir, "y_valid.csv")).to_numpy()[:,1].astype(np.int32)

In [4]:
test_x = pd.read_csv(os.path.join(data_dir, "x_test.csv")).to_numpy()

print(test_x.shape)

(600, 784)


Count number of each class

In [5]:
dress_count_tr = np.count_nonzero(train_y == 3)
dress_count_va = np.count_nonzero(valid_y == 3)
pullover_count_tr = np.count_nonzero(train_y == 2)
pullover_count_va = np.count_nonzero(valid_y == 2)
top_count_tr = np.count_nonzero(train_y == 0)
top_count_va = np.count_nonzero(valid_y == 0)
trouser_count_tr = np.count_nonzero(train_y == 1)
trouser_count_va = np.count_nonzero(valid_y == 1)
sandal_count_tr = np.count_nonzero(train_y == 5)
sandal_count_va = np.count_nonzero(valid_y == 5)
sneaker_count_tr = np.count_nonzero(train_y == 7)
sneaker_count_va = np.count_nonzero(valid_y == 7)

print(dress_count_tr)
print(dress_count_va)
print(pullover_count_tr)
print(pullover_count_va)
print(top_count_tr)
print(top_count_va)
print(trouser_count_tr)
print(trouser_count_va)
print(sandal_count_tr)
print(sandal_count_va)
print(sneaker_count_tr)
print(sneaker_count_va)

400
100
100
100
1
100
1
100
800
100
800
100


Build base model

In [6]:
base_mlp = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes=(800,),
    activation='relu',
    solver='lbfgs',
    max_iter=1000,
    random_state=1
)

# activation, solver, hidden layer, alpha

Create custom splitter for grid search

In [7]:
# custom splitter code

xall_LF = np.vstack([train_x, valid_x])
yall_L = np.hstack([train_y, valid_y])

valid_indicators_L = np.hstack([
    -1 * np.ones(train_y.size),
    0  * np.ones(valid_y.size),
    ])

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

Create Grid Searcher

In [8]:
mlp_hyperparam_grid = dict(
    hidden_layer_sizes=[(8,), (16,), (32,), (64,), (128,), (256,)],
    alpha=np.logspace(-4, 4, 9),
    random_state=[i for i in range(4)],
    max_iter=[10, 100, 1000, 10000]
)

In [9]:
mlp_grid_searcher = sklearn.model_selection.GridSearchCV(
    base_mlp, mlp_hyperparam_grid, scoring='balanced_accuracy', 
    cv=my_splitter, return_train_score=True, refit=False)

Fit grid searcher

In [55]:

mlp_grid_searcher.fit(xall_LF, yall_L)


c:\Users\gmcgu\micromamba\envs\cs135_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\gmcgu\micromamba\envs\cs135_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\gmcgu\micromamba\envs\cs135_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=MLPClassifier(hidden_layer_sizes=(800,),
                                     max_iter=10000000, random_state=1,
                                     solver='lbfgs'),
             param_grid={'alpha': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
                         'hidden_layer_sizes': [(8,), (16,), (32,), (64,),
                                                (128,), (256,), (512,),
                                                (1024,)],
                         'random_state': [0, 1, 2, 3]},
             refit=False, return_train_score=True, scoring='balanced_accuracy')

Get and format the Grid Search Results

In [56]:
mlp_gsearch_results_df = pd.DataFrame(mlp_grid_searcher.cv_results_).copy()

# Rearrange row order so it is easy to skim
mlp_gsearch_results_df.sort_values(by='rank_test_score', inplace=True)
print(mlp_gsearch_results_df[['split0_test_score', 'rank_test_score', 'param_hidden_layer_sizes', 'param_alpha']])

     split0_test_score  rank_test_score param_hidden_layer_sizes param_alpha
146           0.781667                1                   (128,)         0.1
286           0.740000                2                  (1024,)      1000.0
154           0.715000                3                   (512,)         0.1
58            0.713333                4                   (512,)      0.0001
122           0.713333                4                   (512,)        0.01
..                 ...              ...                      ...         ...
238           0.318333              341                    (64,)       100.0
78            0.318333              341                    (64,)       0.001
199           0.308333              350                    (16,)        10.0
71            0.288333              351                    (16,)       0.001
96            0.275000              352                     (8,)        0.01

[352 rows x 4 columns]


In [10]:
print(mlp_grid_searcher.best_params_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [62]:
mlp_gsearch_results_df.to_csv("my_results.txt") 

Add dupes into training data

In [11]:
train_x_dups = train_x
train_y_dups = train_y

size_tr = train_y_dups.shape[0]
for i in range(size_tr):
        # if it is a dress
        if train_y_dups[i] == 3:
                train_y_dups = np.append(train_y_dups, train_y_dups[i])
                train_x_dups = np.vstack([train_x_dups, train_x_dups[i]])
        elif train_y_dups[i] == 2:
                for x in range(7):
                        train_y_dups = np.append(train_y_dups, train_y_dups[i])
                        train_x_dups = np.vstack([train_x_dups, train_x_dups[i]])
        elif train_y[i] == 0 or train_y[i] == 1:
                for x in range(799):
                        train_y_dups = np.append(train_y_dups, train_y_dups[i])
                        train_x_dups = np.vstack([train_x_dups, train_x_dups[i]])

In [12]:
dress_count_tr = np.count_nonzero(train_y_dups == 3)
pullover_count_tr = np.count_nonzero(train_y_dups == 2)
top_count_tr = np.count_nonzero(train_y_dups == 0)
trouser_count_tr = np.count_nonzero(train_y_dups == 1)
sandal_count_tr = np.count_nonzero(train_y_dups == 5)
sneaker_count_tr = np.count_nonzero(train_y_dups == 7)

print(dress_count_tr)
print(pullover_count_tr)
print(top_count_tr)
print(trouser_count_tr)
print(sandal_count_tr)
print(sneaker_count_tr)

800
800
800
800
800
800


In [13]:
print(train_x_dups.shape)

(4800, 784)


In [14]:
xall_LF = np.vstack([train_x_dups, valid_x])
yall_L = np.hstack([train_y_dups, valid_y])

valid_indicators_L = np.hstack([
    -1 * np.ones(train_y_dups.size),
    0  * np.ones(valid_y.size),
    ])

my_dup_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

In [18]:
mlp_dupe_hyperparam_grid = dict(
    hidden_layer_sizes=[(8,), (16,), (32,), (64,), (128,), (256,)],
    alpha=np.logspace(-4, 4, 9),
    random_state=[i for i in range(4)],
    max_iter=[10, 100, 1000, 10000]
)

In [19]:
mlp_dupe_grid_searcher = sklearn.model_selection.GridSearchCV(
    base_mlp, mlp_dupe_hyperparam_grid, scoring='balanced_accuracy', 
    cv=my_dup_splitter, return_train_score=True, refit=False)

In [20]:
mlp_dupe_grid_searcher.fit(xall_LF, yall_L)

/Users/georgemcgurkin/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/georgemcgurkin/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/georgemcgurkin/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/neural_network/_multila

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=MLPClassifier(hidden_layer_sizes=(800,), max_iter=1000,
                                     random_state=1, solver='lbfgs'),
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04]),
                         'hidden_layer_sizes': [(8,), (16,), (32,), (64,),
                                                (128,), (256,)],
                         'max_iter': [10, 100, 1000, 10000],
                         'random_state': [0, 1, 2, 3]},
             refit=False, return_train_score=True, scoring='balanced_accuracy')

In [21]:
mlp_dupes_gsearch_results_df = pd.DataFrame(mlp_dupe_grid_searcher.cv_results_).copy()

# Rearrange row order so it is easy to skim
mlp_dupes_gsearch_results_df.sort_values(by='rank_test_score', inplace=True)
print(mlp_dupes_gsearch_results_df[['split0_test_score', 'rank_test_score', 'param_hidden_layer_sizes', 'param_alpha']])

     split0_test_score  rank_test_score param_hidden_layer_sizes param_alpha
724           0.825000                1                    (64,)      1000.0
549           0.791667                2                   (128,)        10.0
678           0.775000                3                     (8,)      1000.0
628           0.763333                4                    (64,)       100.0
551           0.763333                4                   (128,)        10.0
..                 ...              ...                      ...         ...
288           0.163333              858                     (8,)         0.1
480           0.163333              858                     (8,)        10.0
96            0.163333              858                     (8,)       0.001
0             0.163333              858                     (8,)      0.0001
786           0.118333              864                    (16,)     10000.0

[864 rows x 4 columns]


In [22]:
mlp_dupes_gsearch_results_df.to_csv("1e.csv")

Make prediction on the test set

In [23]:
# mapping of classification id to object name
classifications = {
    3 : "dress",
    2 : "pullover",
    0 : "top",
    1 : "trouser",
    5 : "sandal",
    7 : "sneaker",
}

In [24]:
best_mlp = base_mlp.set_params(**mlp_dupe_grid_searcher.best_params_)

In [25]:
best_mlp.fit(xall_LF, yall_L)

/Users/georgemcgurkin/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1000.0, hidden_layer_sizes=(64,), max_iter=100,
              random_state=0, solver='lbfgs')

In [88]:
yhat_test = best_mlp.predict(test_x)

In [92]:
predictions = [classifications[yhat] for yhat in yhat_test]
np.savetxt('yhat_test.txt', predictions, delimiter='\n', fmt='%s')